In [17]:
from typing import List, Dict, Tuple, Set
import numpy as np
import time
from dataclasses import dataclass
import time
from collections import deque
import pandas as pd
from pathlib import Path
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup

# CHROME_WEBDRIVER_PATH = "/home/fabrice/Documents/PROGRAMMING/ChromeWebDrive/chromedriver"
CHROME_WEBDRIVER_PATH = "/usr/lib/chromium-browser/chromedriver"

# For retries
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Local files.
import constants as c
from website_crawler import WebCrawler

In [36]:
%pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 2.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 2.9 MB/s eta 0:00:003.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [54]:
%pip install html5lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 1.1 MB/s eta 0:00:001.1 MB/s eta 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Data Grabber

In [6]:
tournament_df = pd.read_csv("results/competition_links.csv")
tournament_df

,Tournament link,Base url,Crawl date,Tournament id,Processed
0,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,9,0
1,https://ergebnisse.dancecomp.de/2022/28-0207_w...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,10,0
2,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,11,0
3,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,12,0
4,https://www.gruen-weiss-aachen.de/files/ergebn...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,13,0
...,...,...,...,...,...
218,https://www.hessen-tanzt.de/media/ht2023/56-13...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,227,0
219,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,228,0
220,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,229,0
221,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,230,0


In [11]:
tournament_df.loc[0:3, 'Processed'] = 1
tournament_df

,Tournament link,Base url,Crawl date,Tournament id,Processed
0,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,9,1
1,https://ergebnisse.dancecomp.de/2022/28-0207_w...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,10,1
2,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,11,1
3,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,12,1
4,https://www.gruen-weiss-aachen.de/files/ergebn...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,13,0
...,...,...,...,...,...
218,https://www.hessen-tanzt.de/media/ht2023/56-13...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,227,0
219,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,228,0
220,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,229,0
221,https://www.tanzsport.de/files/tanzsport/ergeb...,https://www.tanzsport.de/de/sportwelt/ergebnisse,2023-07-20,230,0


In [100]:
# Select the links and the ids of unprocessed tournaments.
unprocessed_tournaments_df = tournament_df[tournament_df['Processed'] == 1][[c.CName.TOUR_LINK.value, 'Tournament id']]

for competition_url, competition_id in unprocessed_tournaments_df.itertuples(index=False):
    print(competition_url)
    print(competition_id)

https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/3le-sen1sstd/index.htm
9
https://ergebnisse.dancecomp.de/2022/28-0207_wdsfopenstdsen3/index.htm
10
https://www.tanzsport.de/files/tanzsport/ergebnisse/2022/dm_hgrsstd/index.htm
11
https://www.tanzsport.de/files/tanzsport/ergebnisse/2019/glinde-sen3sstd/index.htm
12


In [101]:
#df.to_csv(c.get_topturnier_df_path(), index=False)

In [122]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(competition_url)

# tournament_title = driver.title
# print(tournament_title)
# html = driver.page_source
# html

In [103]:
# # From the tournament link - extract the competition links.
# regexp = re.compile('src="(.*)"')
# menu_url = regexp.search(html).group(1)
# menu_url

'menu.htm#id=3'

In [126]:
# competition_menue_links = []
# for x in range(10):
#     competition_menue_links.append(competition_url.replace('index.htm', f"{menu_url}{x}"))

# print(competition_menue_links)
competition_menue_links = [competition_url.replace('index.htm', "menu.htm#id=2")]

In [127]:
df_tables = []
for l in competition_menue_links:
    driver.get(l)
        
    html = driver.page_source
    df_tables.append(pd.read_html(html))

In [129]:
if len(df_tables) == 1:
    df_tables = df_tables[0]

len(df_tables)

14

In [175]:
driver.close()

In [130]:
competition_name = df_tables[0][0][0]
competition_name

'DTV-Ranglistenturnier - Sen III S St - Glinde'

In [131]:
general_information = df_tables[1]
general_information

,0,1
0,Veranstalter:,HATV
1,Ausrichter:,Tanzsportverband Schleswig-Holstein e.V.
2,Turnierleiter:,Jes Christophersen Tanzsportfreunde Phoenix Lü...
3,Beisitzer:,Heinrich Dunker TSA d. TSV Glinde 1930
4,Protokoll:,Henning Lange Tanzsportfreunde Phoenix Lütjenb...
5,Turnierbüro:,John Alberts EDV-Team Peter Lorenzen TSA d. TS...
6,Wertungsrichter:,NaN
7,D:,Peter Grell RRC Homburg
8,E:,Pavel Kurgan Grün-Gold-Casino Wuppertal
9,F:,Carsten Lenz 1. Tanzsport Zentrum Freising


In [171]:
split_id = np.where(general_information.iloc[:, 1].isnull().values == True)[0][0]
general_info = general_information.iloc[:split_id, 1]
organiser, host, tournament_director, assessors, protocol, music = general_info
adjudicators = list(general_information.iloc[split_id+1:, 1])
adjudicator_ids = [x[:1] for x in list(general_information.iloc[split_id+1:, 0])]
adjudicators = dict(zip(adjudicator_ids, adjudicators))

In [172]:
print(organiser)
print(host)
print(tournament_director)
print(assessors)
print(protocol)
print(music)
print()
print(adjudicators)

HATV
Tanzsportverband Schleswig-Holstein e.V.
Jes Christophersen Tanzsportfreunde Phoenix Lütjenburg
Heinrich Dunker TSA d. TSV Glinde 1930
Henning Lange Tanzsportfreunde Phoenix Lütjenburg Daniel Randhawa Club Saltatio Hamburg Swen Rist Tanzclub Concordia Lübeck
John Alberts EDV-Team Peter Lorenzen TSA d. TSV Kronshagen 1924 Daniel Randhawa Club Saltatio Hamburg Swen Rist Tanzclub Concordia Lübeck Henning Lange Tanzsportfreunde Phoenix Lütjenburg

{'D': 'Peter Grell RRC Homburg', 'E': 'Pavel Kurgan Grün-Gold-Casino Wuppertal', 'F': 'Carsten Lenz 1. Tanzsport Zentrum Freising', 'G': 'Robert Panther TC Blau-Orange Wiesbaden', 'H': 'Uwe Schadock TSA d. SV Blau Gelb 90 Sonnewalde', 'I': 'Thomas Fürmeyer Tanzsportgemeinschaft Creativ Norderstedt', 'J': 'Tobias Thiele Tanz-Turnier-Club Savoy Norderstedt'}


In [166]:
x0 = tuple(z[0:2])
x1 = " ".join(z[2:])
print(x0)
print(x1)

('Peter', 'Grell')
RRC Homburg


In [174]:
adjudicators_id_club = dict()
for adjudicator_id, adjudicator_and_club in adjudicators.items():
    ac = adjudicator_and_club.split(" ")
    adjudicator, club = tuple(ac[0:2]), " ".join(ac[2:])
    
    adjudicators_id_club[adjudicator_id] = tuple([adjudicator, club])
    
adjudicators_id_club

{'D': (('Peter', 'Grell'), 'RRC Homburg'),
 'E': (('Pavel', 'Kurgan'), 'Grün-Gold-Casino Wuppertal'),
 'F': (('Carsten', 'Lenz'), '1. Tanzsport Zentrum Freising'),
 'G': (('Robert', 'Panther'), 'TC Blau-Orange Wiesbaden'),
 'H': (('Uwe', 'Schadock'), 'TSA d. SV Blau Gelb 90 Sonnewalde'),
 'I': (('Thomas', 'Fürmeyer'), 'Tanzsportgemeinschaft Creativ Norderstedt'),
 'J': (('Tobias', 'Thiele'), 'Tanz-Turnier-Club Savoy Norderstedt')}

In [260]:
adjudicators_df = pd.DataFrame(columns=['Surname','Name','Club'])

In [261]:
competition_id = 12

In [262]:
# Add adjudicators which are unknown (defined as surname, name, club triple).
for _, adjudicator_data in adjudicators_id_club.items():
    name, surname = adjudicator_data[0]
    club = adjudicator_data[1]
    
    # Check if adjudicator is already known:    
    x = adjudicators_df.loc[(adjudicators_df['Surname'] == surname) & (adjudicators_df['Name'] == name) & (adjudicators_df['Club'] == club)]
    # Only add, if not already known.
    if len(x) == 0:   
        new_row = pd.DataFrame({'Surname':surname,'Name':name,'Club':club}, index=[0])
        adjudicators_df = pd.concat([new_row,adjudicators_df.loc[:]]).reset_index(drop=True)

adjudicators_df

,Surname,Name,Club
0,Thiele,Tobias,Tanz-Turnier-Club Savoy Norderstedt
1,Fürmeyer,Thomas,Tanzsportgemeinschaft Creativ Norderstedt
2,Schadock,Uwe,TSA d. SV Blau Gelb 90 Sonnewalde
3,Panther,Robert,TC Blau-Orange Wiesbaden
4,Lenz,Carsten,1. Tanzsport Zentrum Freising
5,Kurgan,Pavel,Grün-Gold-Casino Wuppertal
6,Grell,Peter,RRC Homburg


In [334]:
couples_df = df_tables[2]
couples_df.columns=couples_df.iloc[0]
couples_df = couples_df[1:]
couples_df

,Nr.,Paar,Club
1,1,Jürgen Abelbeck / Birgit Abelbeck,Tanzsportclub Schneverdingen
2,2,Francesco Allegrino / Agata Di Paola,TSA d. MTV Vorsfelde 1862
3,3,Theodor Arlt / Margarete Arlt,TSC Rot-Weiss Viernheim
4,4,Dr. Gerhard Baier / Ingrid Cloos-Baier,TC Brillant Berlin
5,5,Hubert Bartzik / Beate Bartzik,Tanz-Club Tusculum Einbeck
...,...,...,...
115,115,Jörg Monse / Beate Monse,TSA d. Sport-Union Annen
116,116,Steffen Kniestedt / Antje Kniestedt,Tanzsportgemeinschaft Creativ Norderstedt
117,117,Andreas Rekowski / Sabine Rekowski,Askania - TSC Berlin
118,118,Michael Schwarz / Elisabeth Schröder,TSA d. Hamburger Sport-Verein


In [339]:
man_lady_names = couples_df['Paar'].str.split(" / ", expand = True)
man_lady_names.rename(columns={0 :'Man', 1 :'Lady'}, inplace=True )
man_lady_names

,Man,Lady
1,Jürgen Abelbeck,Birgit Abelbeck
2,Francesco Allegrino,Agata Di Paola
3,Theodor Arlt,Margarete Arlt
4,Dr. Gerhard Baier,Ingrid Cloos-Baier
5,Hubert Bartzik,Beate Bartzik
...,...,...
115,Jörg Monse,Beate Monse
116,Steffen Kniestedt,Antje Kniestedt
117,Andreas Rekowski,Sabine Rekowski
118,Michael Schwarz,Elisabeth Schröder


In [340]:
couples_df = couples_df.drop(columns=['Paar'])
couples_df = couples_df.join(x)
couples_df = couples_df[['Nr.', 'Man', 'Lady', 'Club']]
couples_df

,Nr.,Man,Lady,Club
1,1,Jürgen Abelbeck,Birgit Abelbeck,Tanzsportclub Schneverdingen
2,2,Francesco Allegrino,Agata Di Paola,TSA d. MTV Vorsfelde 1862
3,3,Theodor Arlt,Margarete Arlt,TSC Rot-Weiss Viernheim
4,4,Dr. Gerhard Baier,Ingrid Cloos-Baier,TC Brillant Berlin
5,5,Hubert Bartzik,Beate Bartzik,Tanz-Club Tusculum Einbeck
...,...,...,...,...
115,115,Jörg Monse,Beate Monse,TSA d. Sport-Union Annen
116,116,Steffen Kniestedt,Antje Kniestedt,Tanzsportgemeinschaft Creativ Norderstedt
117,117,Andreas Rekowski,Sabine Rekowski,Askania - TSC Berlin
118,118,Michael Schwarz,Elisabeth Schröder,TSA d. Hamburger Sport-Verein


In [379]:
finals_df = df_tables[3]
# Drop the 'Finals'-row.
finals_df = finals_df[1:]
# Set the first row as headder, and drop it.
finals_df.columns = finals_df.iloc[0]
finals_df = finals_df[1:]
finals_df

1,Platz,Paar/Club,LW,TG,WW,SF,QU,PZ
2,1.,Thomas Schmidt / Susanne Schmidt (86) Schwarz-...,1121121 1.0,1111221 1.0,1131521 1.0,1111521 1.0,1231322 2.0,6.0
3,2.,Knut Möller / Iris Möller (64) TTC Rot-Weiß Fr...,3233342 2.0,3233342 2.0,3212342 2.0,3223243 2.0,3122131 1.0,9.0
4,3.,Clemens Wießner-Drude / Nataly Wießner (105) B...,4412265 3.0,4322165 3.0,5423164 3.0,5332164 3.0,5313265 3.0,15.0
5,4.,Andreas Reumann / Annette Reumann (76) Tanz-Tu...,5346613 4.0,5446613 4.0,4356613 4.0,4456612 4.0,4446613 4.0,20.0
6,5.,Nils Goral / Anja Goral (29) TSA im VfL Pinneberg,2654436 5.0,2564436 5.0,2565235 5.0,2564335 5.0,2564544 5.0,25.0
7,6.,Michael Sipek / Susanne Sipek (90) TSC Metropo...,6565554 6.0,6655554 6.0,6644456 6.0,6645456 6.0,6655456 6.0,30.0


In [380]:
# Reduce the couple data to their number.
tmp_df = finals_df['Paar/Club'].str.extract(r'(\(\d*\))', expand=True)        
tmp_df.rename(columns={0 :'Nr.'}, inplace=True)
finals_df = finals_df.drop(columns=['Paar/Club'])
finals_df = finals_df.join(tmp_df)

# Separate the dance rankings and their majority.
dance_cols = finals_df.columns[1:-2]
dance_dfs = []
for c in dance_cols:
    tmp_df = finals_df[c].str.split(r'(\d*\s)', expand=True)
    tmp_df = tmp_df.drop(tmp_df.columns[0],axis=1)
    tmp_df.rename(columns={1 :c, 2: f"{c}_majority"}, inplace=True)
    dance_dfs.append(tmp_df)
    finals_df = finals_df.drop(columns=[c])
    finals_df = finals_df.join(tmp_df)

finals_df

,Platz,PZ,Nr.,LW,LW_majority,TG,TG_majority,WW,WW_majority,SF,SF_majority,QU,QU_majority
2,1.,6.0,(86),1121121,1.0,1111221,1.0,1131521,1.0,1111521,1.0,1231322,2.0
3,2.,9.0,(64),3233342,2.0,3233342,2.0,3212342,2.0,3223243,2.0,3122131,1.0
4,3.,15.0,(105),4412265,3.0,4322165,3.0,5423164,3.0,5332164,3.0,5313265,3.0
5,4.,20.0,(76),5346613,4.0,5446613,4.0,4356613,4.0,4456612,4.0,4446613,4.0
6,5.,25.0,(29),2654436,5.0,2564436,5.0,2565235,5.0,2564335,5.0,2564544,5.0
7,6.,30.0,(90),6565554,6.0,6655554,6.0,6644456,6.0,6645456,6.0,6655456,6.0


In [415]:
all_results = df_tables[4]
# Drop the 'Finals'-row.
all_results = all_results[1:]
# Set the first row as headder, and drop it.
all_results.columns = all_results.iloc[0]
all_results = all_results[1:]

all_results.columns.values[0] = 'Platz'
all_results.columns.values[1] = 'Paar'
all_results.columns.values[2] = 'Club'
all_results = all_results.drop('Club', axis=1)
tmp_df = all_results['Paar'].str.extract(r'(\(\d*\))', expand=True)    
tmp_df.rename(columns={0 :'Nr.'}, inplace=True)
all_results = all_results.drop('Paar', axis=1)
all_results = all_results.join(tmp_df)
all_results

,Platz,Nr.
2,7.,(39)
3,8.,(41)
4,9.- 11.,(9)
5,9.- 11.,(43)
6,9.- 11.,(57)
...,...,...
101,95.- 99.,(65)
102,95.- 99.,(81)
103,100.,(98)
104,101.,(70)


In [382]:
all_results = df_tables[5]
all_results

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,Platz,Paar Club,Nr,R,Langsamer Walzer,Langsamer Walzer,Langsamer Walzer,Langsamer Walzer,Langsamer Walzer,Langsamer Walzer,...,Quickstep,Quickstep,Quickstep,Quickstep,Quickstep,Quickstep,Quickstep,Quickstep,QS,Summe
1,Platz,Paar Club,Nr,R,DPeter Grell,EPavel Kurgan,FCarsten Lenz,GRobert Panther,HUwe Schadock,IThomas Fürmeyer,...,HUwe Schadock,IThomas Fürmeyer,JTobias Thiele,Su,Summe,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.,Thomas Schmidt / Susanne Schmidt Schwarz-Rot-C...,86,F 5 4 3 2 1,1 x x x x x,1 x x x x x,2 x x x x x,1 x x x x x,1 x x x x x,2 x x x x x,...,3 x x x x x,2 x x x x x,2 x x x x x,"2,0 7 7 6 7 7",6.0 35 35 34 35 34,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,101.,Bernd Pförtner / Gerlinde Pförtner Tanzsportcl...,70,1,-,-,-,-,-,-,...,-,-,x,2,5,NaN,NaN,NaN,NaN,NaN
204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,102.,Matthias Sternberg / Iris Sternberg Tanzsportc...,95,1,-,-,-,-,-,-,...,-,-,-,1,3,NaN,NaN,NaN,NaN,NaN


In [70]:
all_results1 = df_tables[6]
all_results1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Nr.,Wertungsrichter,Wertungsrichter,Wertungsrichter,Wertungsrichter,Wertungsrichter,Wertungsrichter,Wertungsrichter,Ausrechnung,Ausrechnung,Ausrechnung,Ausrechnung,Ausrechnung,Ausrechnung,Platz,Summe
1,Nr.,"AFunda, Silke","BVoß, Claudia","CRohne, Madeleine","DGräf, Monika","EDiefert, Bernd","FEggers, Kai","GPernat, Heinz",1.,1.-2.,1.-3.,1.-4.,1.-5.,1.-6.,Platz,Summe
2,NaN,Samba,Samba,Samba,Samba,Samba,Samba,Samba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21Mathias Beckmann / Jennifer Breising,3,3,5,5,5,4,3,0,0,3,4,-,-,50,50
4,22Björn Cremer / Tabea Mayerhoff,5,2,6,2,3,3,5,0,2,4,-,-,-,30,30
5,25Dominik Häring / Julia Schulze,1,6,4,6,4,1,4,2,2,2,5,-,-,40,40
6,27Arno Klöcker / Juliane Braun,4,4,1,3,2,2,2,1,4,-,-,-,-,20,20
7,30Karabo Morake / Nora Isabelle Mergner,2,1,2,1,1,5,1,4,-,-,-,-,-,10,10
8,34Dominic Thutewohl / Mariann Thutewohl,6,5,3,4,6,6,6,0,0,1,2,3,7,60,60
9,NaN,Cha Cha,Cha Cha,Cha Cha,Cha Cha,Cha Cha,Cha Cha,Cha Cha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
all_results2 = df_tables[7]
all_results2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Anzahl Teilnehmer: 10,Startnummer,Startnummer,Startnummer,Startnummer,Startnummer,Startnummer,Startnummer,Startnummer,Startnummer,Startnummer,Startnummer,Startnummer,Startnummer,Startnummer,Startnummer
1,Wertungsrichter,21Mathias Beckmann / Jennifer Breising,22Björn Cremer / Tabea Mayerhoff,23Fabian Geukens / Julia Mindermann,25Dominik Häring / Julia Schulze,27Arno Klöcker / Juliane Braun,29Kai Mielke / Tanja Mielke,30Karabo Morake / Nora Isabelle Mergner,32Thomas Renkel / Saskia Beck,34Dominic Thutewohl / Mariann Thutewohl,36Stephan Wirth / Elisabeth Metzger,NaN,NaN,NaN,NaN,NaN
2,"A) Funda, Silke B) Voß, Claudia C) Rohne, Made...",4 5 4 5 5 4 5,1 5 5 5 5 4 1,2 1 1 0 0 5 2,5 4 5 4 4 4 2,5 5 5 5 5 5 5,1 0 0 0 5 3 3,5 5 5 5 5 0 5,2 1 0 0 1 1 4,5 4 5 5 0 0 0,0 0 0 1 0 4 3,NaN,NaN,NaN,NaN,NaN
3,Ergebnis der Vorrunde,32,26,11,28,35,12,30,9,19,8,NaN,NaN,NaN,NaN,NaN
4,Qualifiziert für die Endrunde,X,X,NaN,X,X,NaN,X,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN
5,Addition Endrunde,230,150,NaN,220,100,NaN,50,NaN,300,NaN,NaN,NaN,NaN,NaN,NaN
6,Startnummer,21Mathias Beckmann / Jennifer Breising,22Björn Cremer / Tabea Mayerhoff,23Fabian Geukens / Julia Mindermann,25Dominik Häring / Julia Schulze,27Arno Klöcker / Juliane Braun,29Kai Mielke / Tanja Mielke,30Karabo Morake / Nora Isabelle Mergner,32Thomas Renkel / Saskia Beck,34Dominic Thutewohl / Mariann Thutewohl,36Stephan Wirth / Elisabeth Metzger,NaN,NaN,NaN,NaN,NaN
7,Platz von Platz bis,5,3,8,4,2,7,1,9,6,10,NaN,NaN,NaN,NaN,NaN
8,Aufstiegspunkte,-,-,-,-,-,-,-,-,-,-,NaN,NaN,NaN,NaN,NaN
